In [590]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Random Forest
from sklearn.ensemble import RandomForestRegressor
# RMSE  (MEN_SQUARED_ERROR IS DEPRECATED)
from sklearn.metrics import root_mean_squared_error



In [591]:
# Load the pkl files
path = '../../data/processed/'
ct = pd.read_pickle(path + 'centrifuga_total.pkl')
cf = pd.read_pickle(path + 'cf.pkl')
ino = pd.read_pickle(path + 'inoculo.pkl')
pino = pd.read_pickle(path + 'preinoculo.pkl')
h = pd.read_pickle(path + 'horas.pkl')
mc = pd.read_pickle(path + 'mov_componentes.pkl')
th = pd.read_pickle(path + 'th.pkl')
of = pd.read_pickle(path + 'of.pkl')
bt = pd.read_pickle(path + 'biorreactor_total.pkl')

In [592]:
cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio                  152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

# Baseline

In [593]:
# What do we drop with dropna
cf_cp = cf.copy()
print(cf_cp.shape)
cf_cp.dropna(inplace=True)
print(cf_cp.shape)
cf_cp.info()

(152, 17)
(138, 17)
<class 'pandas.core.frame.DataFrame'>
Index: 138 entries, 4 to 151
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    138 non-null    object             
 1   orden_encadenado_cf     138 non-null    int64              
 2   lote_parental_cf        138 non-null    object             
 3   id_bio                  138 non-null    object             
 4   f_h_inicio_cf           138 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              138 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         138 non-null    float64            
 7   turb_inicio_cultivo_cf  138 non-null    float64            
 8   turb_fin_cultivo_cf     138 non-null    float64            
 9   viab_final_cultivo_cf   138 non-null    float64            
 10  id_centr                138 non-null    object             
 11  centr_1_turb_cf         138 no

In [594]:
# Fit a classifier with cf data
# Split the data using train_test_splitç
# Cf cpy
cf_cp = cf.copy()
# Drop NA
#cf_cp.dropna(inplace=True)
# Drop the target
target = cf_cp['producto_1_cf'].copy()
# Drop the target
cf_cp.drop('producto_1_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
cf_cp.drop('producto_2_cf', axis=1, inplace=True)
# Drop datetimes
cf_cp.drop(['f_h_inicio_cf', 'f_h_fin_cf'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
cf_cp.drop(['lote', 'lote_parental_cf', 'id_bio', 'id_centr'], axis=1, inplace=True)
#cf_cp.drop(['lote', 'id_centr'], axis=1, inplace=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(cf_cp, target, test_size=0.2, random_state=42)



In [595]:
# Fit the model
# Random Forest
def fit_predict_rf(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(n_estimators=300, random_state=20)
    rf.fit(X_train, y_train)
    # Predict
    y_pred = rf.predict(X_test)
    # RMSE
    rmse = root_mean_squared_error(y_test, y_pred)
    print(f'RMSE: {rmse}')
fit_predict_rf(X_train, X_test, y_train, y_test)

RMSE: 211.22088445376417


In [596]:
# Perform cross validation, with 
from sklearn.model_selection import cross_val_score
def cross_val_rf(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=100, random_state=20)
    scores = cross_val_score(rf, X_train, y_train, cv=15, scoring='neg_root_mean_squared_error')
    print(f'Cross val scores: {scores}')
cross_val_rf(cf_cp, target)

Cross val scores: [-242.23337328 -179.76189859 -199.53043999 -217.6108491  -342.34952746
 -274.85749766 -292.51523283 -320.1172982  -334.13935943 -410.84866335
 -406.78687237 -234.77269544 -248.40865932 -373.49785761 -216.65181894]


In [597]:
from lazypredict.supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np



#X, y = shuffle(boston.data, boston.target, random_state=13)
#X = X.astype(np.float32)

#offset = int(X.shape[0] * 0.9)

#X_train, y_train = X[:offset], y[:offset]
#X_test, y_test = X[offset:], y[offset:]

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:00<00:00, 104.00it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000014 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 121, number of used features: 8
[LightGBM] [Info] Start training from score 1672.990337
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
RandomForestRegressor,0.05,0.34,210.67,0.05
LassoLarsIC,0.03,0.32,212.91,0.00
ElasticNet,0.03,0.32,213.67,0.00
ElasticNetCV,0.03,0.32,213.76,0.02
BayesianRidge,0.02,0.32,213.81,0.00
LassoLarsCV,0.02,0.31,214.17,0.01
LarsCV,0.02,0.31,214.17,0.01
LassoCV,0.02,0.31,214.33,0.03
LinearRegression,0.02,0.31,214.79,0.00


In [598]:
# Extract the most important features
def feat_importance(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=300, random_state=20)
    rf.fit(X_train, y_train)
    # Extract the feature importances
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                       index = X_train.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)
feat_importance(cf_cp, target)

                        importance
turbidez_diff_cf              0.22
turb_fin_cultivo_cf           0.16
centr_2_turb_cf               0.14
centr_1_turb_cf               0.12
viab_final_cultivo_cf         0.10
dur_cf                        0.10
vol_ino_util_cf               0.09
turb_inicio_cultivo_cf        0.07
orden_encadenado_cf           0.01


In [599]:
# Can we predict the product 2?
# Fit a classifier with cf data
# Split the data using train_test_splitç
# Cf cpy
cf_cp = cf.copy()
# Drop NA
#cf_cp.dropna(inplace=True)
# Drop the target
target = cf_cp['producto_2_cf'].copy()
# Drop the target
cf_cp.drop('producto_2_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
cf_cp.drop('producto_1_cf', axis=1, inplace=True)
# Drop datetimes
cf_cp.drop(['f_h_inicio_cf', 'f_h_fin_cf'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
cf_cp.drop(['lote', 'lote_parental_cf', 'id_bio', 'id_centr'], axis=1, inplace=True)
#cf_cp.drop(['lote', 'id_centr'], axis=1, inplace=True)
# dur_cf from timedelta to int

# Split the data
X_train, X_test, y_train, y_test = train_test_split(cf_cp, target, test_size=0.2, random_state=42)
# Fit the model
fit_predict_rf(X_train, X_test, y_train, y_test)
cross_val_rf(cf_cp, target)


RMSE: 1.0413941117471575
Cross val scores: [-2.36141555 -0.79761546 -1.25516046 -0.86751834 -1.69908982 -0.86982418
 -0.87015684 -1.13193536 -1.38329973 -1.24595968 -1.17917768 -1.32266258
 -1.21073274 -1.08662891 -0.92426831]


In [600]:
# Print std and mean of the target
print(f'Target std: {target.std()}')

Target std: 1.4079731610848654


In [601]:
# lazy predict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models


100%|██████████| 42/42 [00:00<00:00, 99.36it/s] 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000013 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 121, number of used features: 8
[LightGBM] [Info] Start training from score 6.194288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
NuSVR,0.25,0.47,0.94,0.00
ExtraTreesRegressor,0.20,0.44,0.97,0.03
SVR,0.15,0.40,1.00,0.00
GradientBoostingRegressor,0.12,0.38,1.01,0.03
RandomForestRegressor,0.11,0.38,1.02,0.05
LGBMRegressor,0.09,0.37,1.03,0.01
AdaBoostRegressor,0.07,0.35,1.04,0.03
HistGradientBoostingRegressor,0.07,0.35,1.04,0.04
KNeighborsRegressor,0.07,0.35,1.04,0.00


# Adding Indexed by Lote

In [602]:
# Shape of the data
print(cf_cp.shape)

(152, 9)


In [603]:
ino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   lote                  167 non-null    object             
 1   id_bio                167 non-null    object             
 2   f_h_inicio_ino        163 non-null    datetime64[ns, UTC]
 3   f_h_fin_ino           163 non-null    datetime64[ns, UTC]
 4   vol_cultivo_ino       163 non-null    float64            
 5   turb_inicio_cultivo   161 non-null    float64            
 6   turb_fin_cultivo      167 non-null    float64            
 7   viab_fin_cultivo_ino  167 non-null    float64            
 8   turb_diff_ino         161 non-null    float64            
 9   dur_ino               163 non-null    float64            
dtypes: datetime64[ns, UTC](2), float64(6), object(2)
memory usage: 13.2+ KB


In [604]:
# Merge ino with cf by lote
ino_cp = ino.copy()
cf_cp = cf.copy()
# Print intersection of columns
print(set(ino_cp.columns).intersection(set(cf_cp.columns)))

{'id_bio', 'lote'}


In [605]:
# Are id_bio for ino and cf the same?
# Print unique values of id_bio
print(ino_cp['id_bio'].unique())
print(cf_cp['id_bio'].unique())
# Rename their id_bio to id_bio_ino and id_bio_cf
ino_cp.rename(columns={'id_bio': 'id_bio_ino'}, inplace=True)
cf_cp.rename(columns={'id_bio': 'id_bio_cf'}, inplace=True)


['13171' '14618' '13172']
['14615' '14616' '13170' '14614' '13169' '14617' '13189']


In [606]:

# Merge
cf_ino = cf_cp.merge(ino_cp, on=['lote'], how='left')
cf_ino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio_cf               152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [1]:
cf_ino.sample(15)

NameError: name 'cf_ino' is not defined

In [608]:
# Now perform lazy predict
# Drop the target
target = cf_ino['producto_1_cf'].copy()
# Drop the target
cf_ino.drop('producto_1_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
cf_ino.drop('producto_2_cf', axis=1, inplace=True)
# Drop datetimes
cf_ino.drop(['f_h_inicio_cf', 'f_h_fin_cf', 'f_h_inicio_ino', 'f_h_fin_ino'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
cf_ino.drop(['lote', 'lote_parental_cf', 'id_bio_cf','id_bio_ino', 'id_centr'], axis=1, inplace=True)


In [609]:

# Split the data
X_train, X_test, y_train, y_test = train_test_split(cf_ino, target, test_size=0.2, random_state=42)
# Fit the model
fit_predict_rf(X_train, X_test, y_train, y_test)


RMSE: 208.12936055078637


In [610]:
# Lazy predict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:00<00:00, 91.27it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000017 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 419
[LightGBM] [Info] Number of data points in the train set: 121, number of used features: 14
[LightGBM] [Info] Start training from score 1672.990337
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
RandomForestRegressor,-0.21,0.40,200.91,0.06
LarsCV,-0.26,0.37,205.32,0.01
LassoLarsCV,-0.26,0.37,205.32,0.01
LassoCV,-0.27,0.37,205.90,0.02
TweedieRegressor,-0.27,0.37,206.21,0.00
ElasticNetCV,-0.27,0.36,206.22,0.02
BayesianRidge,-0.29,0.35,207.92,0.00
GammaRegressor,-0.29,0.35,208.06,0.00
LassoLarsIC,-0.30,0.35,208.30,0.00


In [611]:
# Perform cross validation
cross_val_rf(cf_ino, target)

Cross val scores: [-237.78676725 -163.65378141 -198.73750085 -220.93553189 -371.99440234
 -311.53703376 -274.83606072 -313.41717772 -357.51794402 -425.75998927
 -419.64786824 -245.96496623 -204.13951701 -379.53200132 -197.29145484]


In [612]:
# Feature importance
feat_importance(cf_ino, target)

                        importance
turbidez_diff_cf              0.19
turb_fin_cultivo_cf           0.13
dur_ino                       0.11
centr_2_turb_cf               0.09
centr_1_turb_cf               0.08
vol_ino_util_cf               0.06
turb_fin_cultivo              0.05
viab_final_cultivo_cf         0.05
dur_cf                        0.05
turb_inicio_cultivo_cf        0.05
vol_cultivo_ino               0.04
turb_diff_ino                 0.04
viab_fin_cultivo_ino          0.04
turb_inicio_cultivo           0.03
orden_encadenado_cf           0.01


In [613]:
# Pino
pino.info()


<class 'pandas.core.frame.DataFrame'>
Index: 284 entries, 0 to 303
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   ph_pino          284 non-null    float64            
 1   turb_pino        284 non-null    float64            
 2   f_h_inicio_pino  284 non-null    datetime64[ns, UTC]
 3   f_h_fin_pino     284 non-null    datetime64[ns, UTC]
 4   dur_pino         284 non-null    float64            
 5   lote             284 non-null    object             
dtypes: datetime64[ns, UTC](2), float64(3), object(1)
memory usage: 15.5+ KB


In [614]:
# Merge pino with cf by lote
pino_cp = pino.copy()
cf_cp = cf.copy()
# Print intersection of columns
print(set(pino_cp.columns).intersection(set(cf_cp.columns)))

{'lote'}


In [615]:
# Merge
cf_pino = cf_cp.merge(pino_cp, on=['lote'], how='left')

In [616]:
cf_pino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    256 non-null    object             
 1   orden_encadenado_cf     256 non-null    int64              
 2   lote_parental_cf        256 non-null    object             
 3   id_bio                  256 non-null    object             
 4   f_h_inicio_cf           256 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              256 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         248 non-null    float64            
 7   turb_inicio_cultivo_cf  256 non-null    float64            
 8   turb_fin_cultivo_cf     256 non-null    float64            
 9   viab_final_cultivo_cf   256 non-null    float64            
 10  id_centr                256 non-null    object             
 11  centr_1_turb_cf         252 non-null    float

In [617]:
# Drop duplicates
cf_pino.drop_duplicates(inplace=True)

In [618]:
cf_pino.sample(15)

,lote,orden_encadenado_cf,lote_parental_cf,id_bio,f_h_inicio_cf,f_h_fin_cf,vol_ino_util_cf,turb_inicio_cultivo_cf,turb_fin_cultivo_cf,viab_final_cultivo_cf,...,centr_2_turb_cf,producto_1_cf,producto_2_cf,dur_cf,turbidez_diff_cf,ph_pino,turb_pino,f_h_inicio_pino,f_h_fin_pino,dur_pino
35,23041,1,nan,14616,2023-05-30 05:20:00+00:00,2023-06-01 05:20:00+00:00,82.80,15.60,76.80,260000000.00,...,21.36,1873.68,7.28,172800,61.20,5.43,33.68,2023-05-28 03:00:00+00:00,2023-05-29 05:30:00+00:00,95400.00
207,24008,1,nan,14615,2024-01-20 07:25:00+00:00,2024-01-22 07:00:00+00:00,82.40,17.12,84.00,172800000.00,...,20.80,1575.44,5.44,171300,66.88,5.36,34.40,2024-01-17 23:30:00+00:00,2024-01-19 06:30:00+00:00,111600.00
6,23024,1,nan,14616,2023-03-28 05:24:00+00:00,2023-03-30 05:23:00+00:00,76.00,16.56,80.80,199200000.00,...,10.40,2161.12,2.80,172740,64.24,5.50,28.32,2023-03-26 03:00:00+00:00,2023-03-27 05:21:00+00:00,94860.00
91,23077,1,nan,13170,2023-09-04 10:00:00+00:00,2023-09-06 10:00:00+00:00,78.40,26.24,70.96,157600000.00,...,20.72,2151.54,9.12,172800,44.72,5.42,30.96,2023-09-02 03:00:00+00:00,2023-09-03 05:45:00+00:00,96300.00
9,23025,1,nan,13170,2023-03-29 05:09:00+00:00,2023-03-31 05:29:00+00:00,77.20,17.76,87.20,199200000.00,...,29.36,2044.72,4.48,174000,69.44,5.52,27.52,2023-03-17 05:00:00+00:00,2023-03-28 05:42:00+00:00,956520.00
45,23046,1,nan,14614,2023-06-13 06:00:00+00:00,2023-06-15 07:15:00+00:00,82.80,14.08,69.68,178400000.00,...,14.64,2060.99,8.72,177300,55.60,5.57,26.40,2023-06-11 03:00:00+00:00,2023-06-12 07:00:00+00:00,100800.00
154,23111,1,nan,14615,2023-11-07 10:30:00+00:00,2023-11-09 08:30:00+00:00,81.60,17.60,74.24,195200000.00,...,24.48,1924.00,6.56,165600,56.64,5.46,46.08,2023-11-04 23:30:00+00:00,2023-11-06 11:00:00+00:00,127800.00
17,23029,1,nan,13170,2023-04-18 05:30:00+00:00,2023-04-20 05:30:00+00:00,80.56,16.96,85.60,262400000.00,...,13.36,2063.68,6.72,172800,68.64,5.52,30.32,2023-04-15 03:30:00+00:00,2023-04-16 04:30:00+00:00,90000.00
127,23095,1,nan,13169,2023-10-09 11:15:00+00:00,2023-10-11 11:00:00+00:00,81.20,13.36,44.32,119200000.00,...,10.40,988.10,5.36,171900,30.96,5.41,33.76,2023-10-06 22:30:00+00:00,2023-10-08 06:00:00+00:00,113400.00
227,24027,1,nan,13170,2024-02-10 06:20:00+00:00,2024-02-12 09:55:00+00:00,83.60,18.72,72.88,156800000.00,...,16.24,1663.20,5.12,185700,54.16,5.49,34.88,2024-02-07 23:30:00+00:00,2024-02-09 06:35:00+00:00,111900.00


In [619]:
# Split the data
#X_train, X_test, y_train, y_test = train_test_split(cf_pino, target, test_size=0.2, random_state=42)
# Split by lote
# Get the unique lotes
lotes = cf_pino['lote'].unique()
target = cf_pino['producto_1_cf'].copy()
# Shuffle the lotes
np.random.shuffle(lotes)
# Split the lotes
train_lotes = lotes[:int(0.8*len(lotes))]
test_lotes = lotes[int(0.8*len(lotes)):]

# Train and test
X_train = cf_pino[cf_pino['lote'].isin(train_lotes)]
y_train = target[cf_pino['lote'].isin(train_lotes)]
X_test = cf_pino[cf_pino['lote'].isin(test_lotes)]
y_test = target[cf_pino['lote'].isin(test_lotes)]



In [620]:
# Fit the model
# Drop the target
#target = cf_pino['producto_1_cf'].copy()
# Drop the target
#cf_pino.drop('producto_1_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
#cf_pino.drop('producto_2_cf', axis=1, inplace=True)
# Drop datetimes
#cf_pino.drop(['f_h_inicio_cf', 'f_h_fin_cf', 'f_h_inicio_pino', 'f_h_fin_pino'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
#cf_pino.drop(['lote', 'lote_parental_cf', 'id_bio', 'id_centr'], axis=1, inplace=True)

# Drop those columns but on the train and test
drop_col = ['lote', 'lote_parental_cf', 'id_bio', 'id_centr',
              'producto_1_cf', 'producto_2_cf','f_h_inicio_cf',
              'f_h_fin_cf', 'f_h_inicio_pino', 'f_h_fin_pino']
X_train.drop(drop_col, axis=1, inplace=True)
X_test.drop(drop_col, axis=1, inplace=True)



In [621]:
# Fit the model
fit_predict_rf(X_train, X_test, y_train, y_test)

RMSE: 264.12263682624643


In [626]:
# Cross val
cross_val_rf(X_train, y_train)

Cross val scores: [-174.99077571  -95.22014819 -224.72007273 -232.98590397 -344.27292293
 -275.50882626 -151.48525287 -259.82852465 -357.1513156  -258.93713637
 -427.86076778 -241.37322026 -261.96364353 -357.282176   -189.30072291]


In [627]:
# Lazy predict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:00<00:00, 80.34it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000023 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 455
[LightGBM] [Info] Number of data points in the train set: 201, number of used features: 12
[LightGBM] [Info] Start training from score 1624.689972
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
BaggingRegressor,-0.14,0.11,271.78,0.01
HuberRegressor,-0.16,0.10,273.36,0.01
BayesianRidge,-0.16,0.10,273.64,0.00
OrthogonalMatchingPursuit,-0.16,0.10,273.91,0.00
OrthogonalMatchingPursuitCV,-0.16,0.10,273.91,0.00
ElasticNet,-0.16,0.10,274.02,0.00
SGDRegressor,-0.17,0.09,274.59,0.00
LassoLarsIC,-0.17,0.09,274.78,0.00
RidgeCV,-0.17,0.09,274.94,0.00


In [494]:
# feat importance
feat_importance(cf_pino, target)

                        importance
turbidez_diff_cf              0.22
dur_pino                      0.14
turb_fin_cultivo_cf           0.13
viab_final_cultivo_cf         0.10
centr_2_turb_cf               0.09
centr_1_turb_cf               0.07
dur_cf                        0.06
vol_ino_util_cf               0.06
turb_inicio_cultivo_cf        0.06
turb_pino                     0.04
ph_pino                       0.02
orden_encadenado_cf           0.00
